# Chapter 20: Isolation Levels and Locking

While MVCC eliminates read locks, write operations and explicit locking remain essential for consistency. This chapter provides a comprehensive examination of PostgreSQL's locking infrastructure, from lightweight row locks to application-level advisory locks, with practical patterns for deadlock avoidance and concurrency control in high-contention environments.

## 20.1 Lock Hierarchy and Granularity

PostgreSQL implements multiple lock levels that operate at different granularities, from entire tables to individual rows.

### 20.1.1 Table-Level Lock Modes

```sql
-- PostgreSQL has 8 table-level lock modes, ordered from least to most restrictive:

-- 1. ACCESS SHARE (SELECT)
-- 2. ROW SHARE (SELECT FOR UPDATE/SHARE)
-- 3. ROW EXCLUSIVE (INSERT, UPDATE, DELETE)
-- 4. SHARE UPDATE EXCLUSIVE (VACUUM, ANALYZE, CREATE INDEX CONCURRENTLY)
-- 5. SHARE (CREATE INDEX)
-- 6. SHARE ROW EXCLUSIVE (ALTER TABLE, triggers)
-- 7. EXCLUSIVE (REFRESH MATERIALIZED VIEW CONCURRENTLY)
-- 8. ACCESS EXCLUSIVE (DROP TABLE, ALTER TABLE, VACUUM FULL, LOCK TABLE)

-- Conflict matrix (simplified):
-- ACCESS SHARE conflicts with: ACCESS EXCLUSIVE
-- ROW SHARE conflicts with: EXCLUSIVE, ACCESS EXCLUSIVE  
-- ROW EXCLUSIVE conflicts with: SHARE, SHARE ROW EXCLUSIVE, EXCLUSIVE, ACCESS EXCLUSIVE
-- SHARE conflicts with: ROW EXCLUSIVE, SHARE UPDATE EXCLUSIVE, SHARE ROW EXCLUSIVE, EXCLUSIVE, ACCESS EXCLUSIVE
-- ACCESS EXCLUSIVE conflicts with: All other locks

-- Explicit table locking:
BEGIN;
LOCK TABLE orders IN SHARE MODE;
-- Now other transactions can read but cannot modify rows until commit
-- Useful for bulk operations requiring stable view

-- ACCESS EXCLUSIVE (most restrictive):
BEGIN;
LOCK TABLE orders IN ACCESS EXCLUSIVE MODE;
-- Blocks all access including SELECT
-- Use only for DDL operations or maintenance

-- Check current table locks:
SELECT 
    l.locktype,
    l.relation::regclass as table_name,
    l.mode,
    l.granted,
    a.usename,
    a.application_name,
    left(a.query, 50) as query_snippet
FROM pg_locks l
JOIN pg_stat_activity a ON l.pid = a.pid
WHERE l.locktype = 'relation'
  AND l.relation::regclass::text = 'orders';
```

### 20.1.2 Row-Level Locking Internals

```sql
-- Row locks are stored in the row header (t_infomask bits), not a separate lock table
-- Types: FOR UPDATE, FOR NO KEY UPDATE, FOR SHARE, FOR KEY SHARE

-- Demonstration of lock strength:
-- Session A:
BEGIN;
SELECT * FROM users WHERE user_id = 1 FOR UPDATE;
-- Sets exclusive lock on row 1

-- Session B attempts:
SELECT * FROM users WHERE user_id = 1 FOR UPDATE;  -- Blocks
SELECT * FROM users WHERE user_id = 1 FOR NO KEY UPDATE;  -- Blocks  
SELECT * FROM users WHERE user_id = 1 FOR SHARE;  -- Blocks
SELECT * FROM users WHERE user_id = 1 FOR KEY SHARE;  -- Blocks
UPDATE users SET name = 'X' WHERE user_id = 1;  -- Blocks
DELETE FROM users WHERE user_id = 1;  -- Blocks

-- Lock compatibility matrix:
-- FOR KEY SHARE: Blocks FOR UPDATE, FOR NO KEY UPDATE
-- FOR SHARE: Blocks FOR UPDATE, FOR NO KEY UPDATE  
-- FOR NO KEY UPDATE: Blocks FOR UPDATE, FOR NO KEY UPDATE, FOR SHARE
-- FOR UPDATE: Blocks all row locks and writes

-- Practical difference between FOR UPDATE and FOR NO KEY UPDATE:
-- FOR UPDATE: Strongest, used when deleting or updating primary key
-- FOR NO KEY UPDATE: Used for standard updates (doesn't block FOR KEY SHARE)
-- This distinction matters for foreign key checks (which use FOR KEY SHARE)

-- Example: Parent/child with FK
-- Updating parent primary key requires FOR UPDATE (blocks FK checks)
-- Updating non-key column uses FOR NO KEY UPDATE (allows FK checks to proceed)
```

## 20.2 Deadlock Detection and Resolution

Deadlocks are inevitable in concurrent systems; PostgreSQL detects and resolves them automatically, but application design must minimize their occurrence.

### 20.2.1 How Deadlock Detection Works

```sql
-- PostgreSQL checks for deadlocks every deadlock_timeout (default 1s)
-- When detected, one transaction is aborted with error code 40P01

-- Creating a deliberate deadlock:
-- Session A:
BEGIN;
UPDATE accounts SET balance = 900 WHERE id = 1;  -- Locks row 1
-- ... wait ...

-- Session B:
BEGIN;
UPDATE accounts SET balance = 800 WHERE id = 2;  -- Locks row 2
UPDATE accounts SET balance = 800 WHERE id = 1;  -- Waits for Session A (row 1)

-- Session A continues:
UPDATE accounts SET balance = 900 WHERE id = 2;  -- Waits for Session B (row 2)
-- DEADLOCK DETECTED
-- ERROR: deadlock detected
-- DETAIL: Process 12345 waits for ShareLock on transaction 67890; blocked by process 12346.
-- HINT: See server log for query details.
-- CONTEXT: while updating tuple (0,1) in relation "accounts"

-- Session A is aborted (ROLLBACK), Session B proceeds
-- Which transaction is aborted? PostgreSQL chooses the one with:
-- 1. Fewer locks held (less work to undo)
-- 2. If equal, random selection
```

### 20.2.2 Deadlock Prevention Patterns

```sql
-- Pattern 1: Consistent Ordering (most important)
-- Always acquire locks in the same order (e.g., by primary key ascending)

-- Bad (potential deadlock):
-- Transfer A->B: Lock A, then B
-- Transfer B->A: Lock B, then A
-- If both happen concurrently: deadlock

-- Good (deadlock-free):
-- Always lock lower ID first
-- Transfer A->B (A=1, B=2): Lock 1, then 2
-- Transfer B->A (B=2, A=1): Lock 1 (A), then 2 (B)
-- Same order = no circular wait

-- Implementation:
BEGIN;
-- Sort IDs to ensure consistent locking order
SELECT * FROM accounts 
WHERE id IN (LEAST($1, $2), GREATEST($1, $2)) 
ORDER BY id 
FOR UPDATE;

-- Pattern 2: Two-Phase Locking with Retry
-- Application code structure:
/*
def transfer_funds(from_id, to_id, amount, max_retries=3):
    for attempt in range(max_retries):
        try:
            with db.transaction():
                # Lock both rows in consistent order
                rows = db.query(
                    "SELECT * FROM accounts WHERE id IN (%s, %s) ORDER BY id FOR UPDATE",
                    min(from_id, to_id), max(from_id, to_id)
                )
                # Verify balances, update both
                db.execute("UPDATE accounts SET balance = balance - %s WHERE id = %s", amount, from_id)
                db.execute("UPDATE accounts SET balance = balance + %s WHERE id = %s", amount, to_id)
                return True
        except DeadlockDetected:
            if attempt == max_retries - 1:
                raise
            time.sleep(random.uniform(0.01, 0.1))  # Exponential backoff
*/

-- Pattern 3: Reduce Lock Contention (optimistic locking)
-- Instead of SELECT FOR UPDATE, use version checking:
BEGIN;
SELECT balance, version FROM accounts WHERE id = 1;
-- Application calculates: new_balance = balance - 100
UPDATE accounts 
SET balance = balance - 100, version = version + 1 
WHERE id = 1 AND version = :previous_version;
-- If rows affected = 0, another transaction modified the row (retry)
COMMIT;
-- No explicit lock held during calculation, shorter critical section
```

## 20.3 Advisory Locks

Advisory locks provide application-level synchronization without consuming table rows, ideal for coordinating external processes or complex business logic.

### 20.3.1 Session vs Transaction Advisory Locks

```sql
-- Two scopes:
-- 1. Session-level: Released when session disconnects (or explicit unlock)
-- 2. Transaction-level: Released at transaction commit/rollback (preferred)

-- Transaction-level (automatic cleanup):
BEGIN;
SELECT pg_advisory_xact_lock(123);  -- Lock ID 123
-- ... critical section ...
COMMIT;  -- Lock automatically released

-- Session-level (manual management):
SELECT pg_advisory_lock(456);  -- Acquire
-- ... work ...
SELECT pg_advisory_unlock(456);  -- Must release manually
-- If session dies, lock is released (cleanup on disconnect)

-- Try-lock (non-blocking):
SELECT pg_advisory_try_xact_lock(789);
-- Returns true if acquired, false if already held by another session
-- Use for optional work: "if nobody else is doing this, I'll do it"

-- Multiple locks:
SELECT pg_advisory_xact_lock(1), pg_advisory_xact_lock(2);
-- Acquires both (order matters for deadlock prevention!)

-- Lock IDs:
-- Use 64-bit integers (or two 32-bit)
-- Convention: Hash of resource type + ID
-- Example: pg_advisory_xact_lock('myapp:job:' || job_id)::bigint - hash to int
-- Or use two int keys: pg_advisory_xact_lock(resource_type_id, resource_id)
```

### 20.3.2 Practical Advisory Lock Patterns

```sql
-- Pattern 1: Queue Processing (Singleton Workers)
-- Ensure only one worker processes a queue at a time:
BEGIN;
SELECT pg_advisory_xact_lock('global_queue_processor'::regclass::oid::int);
-- Check if already processing, if not:
UPDATE jobs SET status = 'processing' WHERE status = 'pending' LIMIT 1;
COMMIT;

-- Pattern 2: Unique Job Execution (prevent duplicate cron jobs)
-- Cron runs every minute, but job might take 5 minutes:
BEGIN;
-- Try to acquire lock for this specific job type
IF pg_advisory_try_xact_lock('daily_report_job') THEN
    -- Execute report generation (takes 5 minutes)
    PERFORM generate_daily_report();
    COMMIT;
    -- Lock held for duration, next cron sees lock busy and skips
ELSE
    ROLLBACK;
    -- Another instance is running, exit silently
END IF;

-- Pattern 3: External Resource Coordination
-- Coordinate with external API rate limits:
BEGIN;
-- Lock based on API endpoint hash
SELECT pg_advisory_xact_lock(hashtext('stripe_api_charge'));
-- Make API call (limited to one concurrent call per endpoint)
-- ... HTTP request ...
INSERT INTO api_calls (endpoint, response) VALUES (...);
COMMIT;
```

## 20.4 Optimistic vs Pessimistic Concurrency Control

Choosing between locking strategies impacts throughput and conflict rates.

### 20.4.1 Pessimistic Locking (SELECT FOR UPDATE)

```sql
-- Strategy: Lock resources before modifying, guaranteeing exclusive access
-- Best for: High contention (many writers), long transactions, financial accuracy

-- Implementation:
BEGIN;
-- Lock inventory rows for specific SKUs
SELECT * FROM inventory 
WHERE sku IN ('SKU-001', 'SKU-002') 
FOR UPDATE;

-- Check stock levels (locked, no one else can modify)
-- Calculate allocations
-- Update inventory
UPDATE inventory SET qty = qty - 5 WHERE sku = 'SKU-001';
UPDATE inventory SET qty = qty - 3 WHERE sku = 'SKU-002';

-- Insert order (foreign keys checked while locks held)
INSERT INTO orders ...
COMMIT;

-- Advantages:
-- 1. No retry logic needed (locks prevent conflicts)
-- 2. Consistent view guaranteed
-- 3. Suitable for complex multi-step business logic

-- Disadvantages:
-- 1. Lock contention reduces concurrency
-- 2. Risk of deadlocks (requires ordering discipline)
-- 3. Locks held for duration of transaction (keep transactions short!)
```

### 20.4.2 Optimistic Locking (Version Control)

```sql
-- Strategy: Check for conflicts only at write time using version numbers
-- Best for: Read-heavy workloads, low contention, short updates

-- Table design:
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    name TEXT,
    price DECIMAL,
    stock INTEGER,
    version INTEGER DEFAULT 1,  -- Optimistic lock column
    updated_at TIMESTAMPTZ
);

-- Application flow:
-- 1. Read product (no lock):
--    SELECT product_id, price, stock, version FROM products WHERE product_id = 1;
--    -- Returns: price=100, stock=50, version=5

-- 2. Application does work (calculates new price, etc.)
--    new_price = calculate_price(100);
--    new_stock = 50 - 1;

-- 3. Update with version check:
UPDATE products 
SET price = 105.00, 
    stock = 49, 
    version = 6,
    updated_at = NOW()
WHERE product_id = 1 
  AND version = 5;  -- Must match what we read!

-- Check rows affected:
-- If 1 row: Success, no conflict
-- If 0 rows: Conflict detected (another user updated version to 6)
--            Application must retry from step 1

-- Advanced: Optimistic with partial field updates
-- If different fields updated, sometimes no conflict (field-level versioning):
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    content TEXT,
    metadata JSONB,
    content_version INTEGER DEFAULT 1,
    metadata_version INTEGER DEFAULT 1
);
-- Update content only checks content_version
-- Update metadata only checks metadata_version
-- Reduces false conflicts
```

### 20.4.3 Hybrid Approaches

```sql
-- Strategy: Optimistic for reads, pessimistic for critical sections
-- Example: E-commerce checkout

BEGIN;
-- Step 1: Optimistic check (no lock, fast)
SELECT price, stock, version FROM products WHERE product_id IN (1, 2);
-- Verify all in stock, calculate totals

-- Step 2: Pessimistic lock only for inventory deduction (critical section)
SELECT * FROM inventory 
WHERE product_id IN (1, 2) 
FOR UPDATE NOWAIT;  -- Fail fast if can't lock

-- Verify stock hasn't changed (optimistic check within pessimistic lock)
-- If stock changed (optimistic failure), rollback and restart

-- Step 3: Guaranteed update (pessimistic ensures no one else changes it)
UPDATE inventory SET stock = stock - qty WHERE product_id = 1;
UPDATE inventory SET stock = stock - qty WHERE product_id = 2;
INSERT INTO order_items ...
COMMIT;

-- This minimizes lock duration (only at final step) while ensuring consistency
```

## 20.5 Monitoring Lock Contention

Proactive monitoring prevents lock escalation and downtime.

### 20.5.1 Real-Time Lock Analysis

```sql
-- View current lock waits:
SELECT 
    blocked_locks.pid AS blocked_pid,
    blocked_activity.usename AS blocked_user,
    blocking_locks.pid AS blocking_pid,
    blocking_activity.usename AS blocking_user,
    blocked_activity.query AS blocked_statement,
    blocking_activity.query AS blocking_statement,
    blocked_activity.wait_event_type AS blocked_wait_type,
    blocked_activity.wait_event AS blocked_wait_event,
    NOW() - blocked_activity.query_start AS blocked_duration
FROM pg_catalog.pg_locks blocked_locks
JOIN pg_catalog.pg_stat_activity blocked_activity 
    ON blocked_activity.pid = blocked_locks.pid
JOIN pg_catalog.pg_locks blocking_locks 
    ON blocking_locks.locktype = blocked_locks.locktype
    AND blocking_locks.database = blocked_locks.database
    AND blocking_locks.relation = blocked_locks.relation
    AND blocking_locks.page = blocked_locks.page
    AND blocking_locks.tuple = blocked_locks.tuple
    AND blocking_locks.virtualxid = blocked_locks.virtualxid
    AND blocking_locks.transactionid = blocked_locks.transactionid
    AND blocking_locks.classid = blocked_locks.classid
    AND blocking_locks.objid = blocked_locks.objid
    AND blocking_locks.objsubid = blocked_locks.objsubid
    AND blocking_locks.pid != blocked_locks.pid
JOIN pg_catalog.pg_stat_activity blocking_activity 
    ON blocking_activity.pid = blocking_locks.pid
WHERE NOT blocked_locks.granted
  AND blocked_activity.query_start < NOW() - interval '5 seconds';

-- Kill blocking query (emergency only):
-- SELECT pg_terminate_backend(blocking_pid);

-- Check for heavyweight locks (table-level):
SELECT 
    l.locktype,
    l.mode,
    l.granted,
    a.usename,
    a.client_addr,
    left(a.query, 100) as query
FROM pg_locks l
JOIN pg_stat_activity a ON l.pid = a.pid
WHERE l.locktype = 'relation'
  AND l.mode = 'AccessExclusiveLock';
-- AccessExclusiveLock on production tables = downtime for that table
```

### 20.5.2 Historical Contention Analysis

```sql
-- Enable statistics collector (postgresql.conf):
-- track_activities = on
-- track_counts = on

-- View lock statistics:
SELECT 
    relname,
    relid,
    pid,
    mode,
    granted
FROM pg_locks l
JOIN pg_class c ON l.relation = c.oid
WHERE relkind = 'r'
ORDER BY relname;

-- Check for lock escalation trends:
-- If row-level locks frequently escalate to table-level, indicates:
-- 1. Missing VACUUM (table bloat causes PostgreSQL to lock more)
-- 2. Too many row locks (increase max_locks_per_transaction)
-- 3. Long transactions holding many locks

-- Detect "idle in transaction" holding locks:
SELECT 
    pid,
    usename,
    state,
    xact_start,
    now() - xact_start as xact_duration,
    wait_event_type,
    wait_event,
    left(query, 100)
FROM pg_stat_activity
WHERE state = 'idle in transaction'
  AND xact_start < now() - interval '1 minute';
-- These hold locks but do no work = concurrency killer
```

## 20.6 Serialization Failures and Handling

Serializable isolation provides the strongest guarantees but requires application-level retry logic.

### 20.6.1 Detecting Serialization Anomalies

```sql
-- Serializable creates predicate locks that track read/write dependencies
-- When potential circular dependency detected, one transaction aborted:

BEGIN ISOLATION LEVEL SERIALIZABLE;
SELECT * FROM accounts WHERE balance > 1000;  -- Predicate lock: balance > 1000
-- ... do work ...
UPDATE accounts SET balance = balance - 100 WHERE account_id = 5;
COMMIT;
-- ERROR: could not serialize access due to read/write dependencies among transactions
-- DETAIL: Reason code: Canceled on identification as a pivot, during commit attempt.
-- HINT: The transaction might succeed if retried.

-- Retry logic is mandatory:
/*
function execute_serializable(transaction_fn, max_retries=3):
    for attempt in 1..max_retries:
        try:
            db.begin(isolation='serializable')
            result = transaction_fn()
            db.commit()
            return result
        except SerializationFailure:
            db.rollback()
            if attempt == max_retries:
                raise
            sleep(random(0.01 * 2^attempt))  -- Exponential backoff
        except DeadlockDetected:
            db.rollback()
            if attempt == max_retries:
                raise
            sleep(random(0.01, 0.1))
*/

-- Serialization failure indicators:
-- 1. High retry rates (>5% indicates excessive contention)
-- 2. Specific tables frequently involved (redesign access patterns)
-- 3. Specific time periods (batch jobs conflicting with OLTP)
```

### 20.6.2 Reducing Serialization Conflicts

```sql
-- Strategy 1: Shrink transaction scope
-- Bad:
BEGIN ISOLATION LEVEL SERIALIZABLE;
SELECT * FROM config;  -- Read config (rarely changes, but creates predicate lock)
-- ... 5 seconds of business logic ...
UPDATE orders SET status = 'shipped';
COMMIT;

-- Good:
-- Read config outside transaction (if immutable for transaction duration)
-- Or use SELECT FOR SHARE (weaker lock) for config if it must be consistent
BEGIN ISOLATION LEVEL SERIALIZABLE;
-- Only put absolutely necessary operations in serializable block
UPDATE orders SET status = 'shipped' WHERE order_id = 123;
COMMIT;

-- Strategy 2: Partition access (avoid overlapping predicate spaces)
-- Instead of all workers processing all orders:
-- Worker 1: order_id % 4 = 0
-- Worker 2: order_id % 4 = 1
-- etc.
-- Predicate locks on disjoint ranges don't conflict

-- Strategy 3: Materialize reads (snapshot materialization)
-- Use REPEATABLE READ instead if write skew not possible for specific workflow
-- Or use explicit locking (FOR UPDATE) to serialize access explicitly
```

## 20.7 Locking Anti-Patterns and Solutions

### 20.7.1 Lock Escalation Triggers

```sql
-- Anti-pattern 1: Long transactions with many updates
BEGIN;
UPDATE row1;
-- ... 1000 more updates ...
UPDATE row1000;
COMMIT;
-- Holds 1000 row locks for duration = high deadlock risk, blocks vacuum

-- Solution: Batch commits
-- Process 100 per transaction, commit, next batch

-- Anti-pattern 2: SELECT FOR UPDATE without WHERE clause
SELECT * FROM large_table FOR UPDATE;
-- Locks every row in table = effectively table lock

-- Solution: Lock only what you need with precise WHERE clause

-- Anti-pattern 3: Unnecessarily high isolation level
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
-- For simple single-row updates = overkill, use READ COMMITTED

-- Anti-pattern 4: Holding locks during external calls
BEGIN;
SELECT * FROM inventory WHERE sku = 'ABC' FOR UPDATE;
-- HTTP call to shipping API (3 seconds)
-- Lock held for 3 seconds = concurrency bottleneck
UPDATE inventory SET ...
COMMIT;

-- Solution: Two-phase (optimistic check, then pessimistic lock only for update)
-- Do HTTP call first, then quick transaction for DB update
-- Or use advisory lock for external coordination (doesn't block row access)

-- Anti-pattern 5: Missing indexes on foreign keys
-- DELETE FROM parent WHERE id = 1;
-- Without index on child.parent_id, must scan entire child table
-- Holds lock on parent while scanning child = long lock duration

-- Solution: Always index foreign keys
CREATE INDEX idx_child_parent ON child(parent_id);
```

### 20.7.2 Queue Processing Locks

```sql
-- Anti-pattern: Skip locked without ordering (unfair processing)
SELECT * FROM jobs 
WHERE status = 'pending' 
FOR UPDATE SKIP LOCKED 
LIMIT 1;
-- Grabs random pending job
-- Problem: Low priority jobs might starve high priority jobs

-- Solution: Ordered skip locked
SELECT * FROM jobs 
WHERE status = 'pending' 
ORDER BY priority DESC, created_at ASC 
FOR UPDATE SKIP LOCKED 
LIMIT 1;
-- Ensures highest priority available job is grabbed

-- Batch queue processing with skip locked:
BEGIN;
SELECT * FROM jobs 
WHERE status = 'pending' 
ORDER BY created_at 
FOR UPDATE SKIP LOCKED 
LIMIT 10;
-- Process 10 at once (reduces transaction overhead)
UPDATE jobs SET status = 'processing' WHERE id IN (...selected ids...);
COMMIT;
```

---

## Chapter Summary

In this chapter, you learned:

1. **Lock Hierarchy**: PostgreSQL uses table-level locks (8 modes from ACCESS SHARE to ACCESS EXCLUSIVE) for DDL and bulk operations, and row-level locks (FOR KEY SHARE, FOR SHARE, FOR NO KEY UPDATE, FOR UPDATE) for DML. Row locks are stored in tuple headers, not a separate lock table.

2. **Deadlock Management**: Deadlocks are detected via wait-for graph analysis every `deadlock_timeout` (default 1s). Prevention requires consistent lock acquisition ordering (always lock resources in the same sequence, e.g., by ID ascending). Applications must implement exponential backoff retry logic for deadlock and serialization failures.

3. **Advisory Locks**: Application-level synchronization using 64-bit integer identifiers. Transaction-level advisory locks (`pg_advisory_xact_lock`) automatically release at commit/rollback, preventing lock leaks. Ideal for queue singletons, cron job coordination, and external API rate limiting without blocking table access.

4. **Concurrency Strategies**: Pessimistic locking (`SELECT FOR UPDATE`) guarantees consistency by blocking conflicts, suitable for high-contention financial operations but reduces throughput. Optimistic locking uses version columns to detect conflicts at write time, better for read-heavy workloads but requires application retry logic. Hybrid approaches minimize lock duration by using optimistic checks for reads and pessimistic locks only for critical writes.

5. **Serialization Handling**: Serializable isolation prevents all anomalies including write skew through predicate locking, but requires mandatory retry logic for serialization failures. High retry rates (>5%) indicate excessive contention requiring transaction scope reduction or access pattern partitioning.

6. **Monitoring**: `pg_locks` joined with `pg_stat_activity` reveals blocking queries. "Idle in transaction" states holding locks are critical performance killers. AccessExclusiveLock on production tables indicates DDL operations blocking all access. Foreign keys without indexes cause table scans during parent updates/deletes, extending lock duration.

**Next:** In Chapter 21, we will explore Sequences and ID Generation—covering SERIAL vs IDENTITY, sequence caching, UUID strategies, and distributed ID generation patterns for high-throughput systems.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. transactions_and_mvcc.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. sequences_and_id_generation.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
